### Tire Wear Classification

**2 Classes:**

**Normal**: including Like New, Normal Wear & Retread

**Irregular**: including Side Wear & Un-specified Wear

In [1]:
### Setup
import fastai
from fastai.data.all import *
from fastai.vision.all import *
from fastai.vision.widgets import *

In [2]:
### CHANGE PATH WHEN DEPLOYMENT ! ###
data_path = Path('/mnt/raid/users/thoang/tyre/')
# data_path = Path('data')

In [3]:
# Data Block
tyres = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=28),
                 get_y=parent_label,
                 item_tfms=Resize(400),
                 batch_tfms=[*aug_transforms(min_scale=0.75, size=224),
                            Normalize.from_stats(*imagenet_stats)])

In [4]:
dls = tyres.dataloaders(data_path/"train", bs=64, number_workers=-1)

In [7]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)

In [8]:
path_export = Path()
learn = learn.load(path_export/'tyre-learn-v2.pkl')

In [9]:
btn_run = widgets.Button(description='Classify')
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [10]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(256, 256))
    pred,pred_idx,probs = learn.predict(img)
    lbl_pred.value = f'Prediction: {str(pred).upper()} | Probability: {probs[pred_idx]*100:.02f}%'

btn_run.on_click(on_click_classify)

In [11]:
VBox([widgets.Label('Select your Tire!'), 
      btn_upload, btn_run, out_pl, lbl_pred])